In [1]:
#引入需要的包裝

import pyvisa
import time

### AFG31101

===============Function List=================

* Set weveform type
* Set frequency
* Set amplitude
* Set output state
* Set amplitude limit
  
若要串接其他儀器 一樣使用class 定義其他儀器的function

In [2]:
class AFG_31101():
    def __init__(self, resource_address, visa_dll=None):
        """
        初始化儀器控制器。
        :param resource_address: 儀器的資源地址。
        :param visa_dll: 使用的 VISA DLL 路徑。如果為 None，則使用系統預設空白。
        """
        self.resource_address = resource_address
        self.visa_dll = visa_dll if visa_dll is not None else ''
        self.rm = pyvisa.ResourceManager(self.visa_dll)
        self.instrument = None
        self.connect_to_instrument()
        
    def connect_to_instrument(self):
         # 建立與儀器的連接
        self.instrument = self.rm.open_resource(self.resource_address)
        print("Connected to instrument.")
        print(self.query_identity())

    def query_identity(self):
         # 查詢並返回儀器的身份識別信息
        if self.instrument:
             return self.instrument.query("*IDN?")
        else:
             raise Exception("Instrument not connected. Please connect first.")

    def close_instrument(self):
        if self.instrument:
            self.instrument.close()
            print("Instrument connection closed.")
        else:
            print("No instrument to close.")

    def set_waveform(self, shape):
        if self.instrument:
            self.instrument.write(f"SOURce1:FUNCtion {shape}")
            time.sleep(0.05)
            print(f"set function shape be {shape}")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def set_frequency(self, freq_magnitude, freq_unit):
        if self.instrument:
            self.instrument.write(f"SOURce1:FREQuency:FIXed {freq_magnitude} {freq_unit}")
            time.sleep(0.05)
            print(f"set frequency be {freq_magnitude}{freq_unit}")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def query_frequency(self):
        if self.instrument:
            self.instrument.write(f"SOURce1:FREQuency:FIXed?")
            time.sleep(0.05)
            return self.instrument.read()
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def set_amplitude(self, amp_magnitude, amp_unit):
        if self.instrument:
            self.instrument.write(f"SOURce1:VOLTage:LEVel:IMMediate:AMPLitude {amp_magnitude}{amp_unit}")
            time.sleep(0.05)
            print(f"set amplitude be {amp_magnitude}{amp_unit}")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def query_amplitude(self):
        if self.instrument:
            self.instrument.write(f"SOURce1:VOLTage:LEVel:IMMediate:AMPLitude?")
            time.sleep(0.05)
            return self.instrument.read()
        else:
            raise Exception("Instrument not connected. Please connect first.")
    
    def output_on(self):
        if self.instrument:
            self.instrument.write(f"OUTPut1:STATe ON")
            time.sleep(0.05)
            print(f"turn output on")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def output_off(self):
        if self.instrument:
            self.instrument.write(f"OUTPut1:STATe OFF")
            time.sleep(0.05)
            print(f"turn output off")
        else:
            raise Exception("Instrument not connected. Please connect first.")

In [3]:
def check_amp_limit(afg, amplitude_limit):
    query_amp = afg.query_amplitude()  # 回傳波形振幅大小
    amp_num = float(query_amp)  # 將回傳振幅大小的字串型態轉為數字型態
    if amp_num >= amplitude_limit:  # 判斷振幅是否大於AMPLITUDE_LIMIT
        print(f'Magnitude of amplitude cannot exceed {amplitude_limit} voltage')
        afg.output_off()  # 將OUTPUT關閉

In [4]:
def main():
    
    # =========================Parmeters===============================
    """
    shape:{SINusoid|SQUare|PULSe|RAMP|PRNoise|DC|SINC|GAUSsian|
    LORentz|ERISe|EDECay|HAVersine|EMEMory[1]|EMEMory2|EFILe} 
    amplitude:
        magnitude:output amplitude
        <units>::=[VPP | VRMS | DBM]
    frequency:
        magnitude:output frequency
        <units>::=[Hz | kHz | MHz]
    output_state:{ON|OFF}
    """
    #設置訊號參數
    shape = 'SINusoid'
    frequency = {'magnitude': '800','unit': 'KHz'}
    amplitude = {'magnitude': '300E-3','unit': 'VPP'}
    output_state = "ON"
    amplitude_limit = 1
    
    #====================Instrument Connect Test===========================
    afg_instrument_address = 'USB0::0x0699::0x0359::C016504::INSTR'
    afg = AFG_31101(afg_instrument_address) #連接AFG_31101
     #若有其他儀器串接下去...

    #============================Auto Test================================= 
    # TO DO:寫For迴圈自動測量的區域
    #設置訊號類
    afg.set_waveform(shape) 
    
    #設置訊號頻率
    afg.set_frequency(frequency['magnitude'],frequency['unit'])
    
    #回傳波形頻率
    query_freq = afg.query_frequency() 
    print(float(query_freq)) #將回傳頻率的字串型態轉為數字型態
    
    #設置訊號振幅
    afg.set_amplitude(amplitude['magnitude'], amplitude['unit']) 

    if output_state == 'ON': #判斷輸出狀態，選擇要操作在哪一個輸出狀態DEF
        afg.output_on()
    else:
        afg.output_off()

    check_amp_limit(afg, amplitude_limit)  # 檢查振幅是否超過限制
        
    # ============================ Close Instrument=================================
    afg.close_instrument() #程式結束將儀器斷連，方便在不同程式檔中切換控制儀器
    
if __name__ == "__main__":
    main()

Connected to instrument.
TEKTRONIX,AFG31101,C016504,SCPI:99.0 FV:1.6.1

set function shape be SINusoid
set frequency be 800KHz
800000.0
set amplitude be 300E-3VPP
turn output on
Instrument connection closed.
